# Usermore soap case study

Guilherme Fernandes e Lauro Solia

In [26]:
from pprint import pprint

import gurobipy
from gurobipy import Model, GRB

import pandas as pd

## 1. Defining the sets

In [27]:
plants = [
    "Covington, KY",
    "New York, NY",
    "Arlington, TX",
    "Long Beach, CA",
]

existing_warehouses = [
    "Atlanta",
    "Boston",
    "Buffalo",
    "Chicago",
    "Cleveland",
    "Davenport",
    "Detroit",
    "Grand Rapids",
    "Greensboro",
    "Kansas City",
    "Baltimore",
    "Memphis",
    "Milwaukee",
    "Orlando",
    "Pittsburgh",
    "Portland",
    "W Sacramento",
    "W Chester",
]

potential_warehouses = [
    "Albuquerque",
    "Billings",
    "Denver",
    "El Paso",
    "Camp Hill",
    "Houston",
    "Las Vegas",
    "Minneapolis",
    "New Orleans",
    "Phoenix",
    "Richmond",
    "St Louis",
    "Salt Lake City",
    "San Antonio",
    "Seattle",
    "Spokane",
    "San Francisco",
    "Indianapolis",
    "Louisville",
    "Columbus",
    "New York",
    "Hartford",
    "Miami",
    "Mobile",
    "Memphis *",
    "Chicago *",
]

# Merge existing + potential
warehouses = existing_warehouses + potential_warehouses

In [28]:
print("Number of plants: ", len(plants))
print("Number of existing warehouses: ", len(existing_warehouses))
print("Number of potential warehouses: ", len(potential_warehouses))
print("Number of total warehouses: ", len(warehouses))
print("Obs.: Each plant is also an existing warehouse")

Number of plants:  4
Number of existing warehouses:  18
Number of potential warehouses:  26
Number of total warehouses:  44
Obs.: Each plant is also an existing warehouse


## 2. Parameters

In [29]:
# read state demands from Figure 1
state_demand = {
    # West region:
    "WA": 32437,  # Washington
    "OR": 31365,  # Oregon
    "CA": 135_116,  # California
    "NV": 16755,  # Nevada
    "AZ": 9063,  # Arizona
    "ID": 7153,  # Idaho
    # Northwest region:
    "UT": 9001,  # Utah
    "MT": 4140,  # Montana
    "ND": 5703,  # North Dakota
    "WY": 1004,  # Wyoming
    "CO": 11147,  # Colorado
    "SD": 1049,  # South Dakota
    "NE": 7347,  # Nebraska
    "KS": 6961,  # Kansas
    "MN": 5633,  # Minnesota
    "IA": 32175,  # Iowa
    "MO": 41680,  # Missouri
    # Southwest region:
    "NM": 3536,  # New Mexico
    "TX": 80438,  # Texas
    "OK": 13517,  # Oklahoma
    "AR": 4910,  # Arkansas
    "LA": 15011,  # Louisiana
    # Midwest region:
    "WI": 37448,  # Wisconsin
    "IL": 72839,  # Illinois
    "MI": 105_181,  # Michigan
    "IN": 43994,  # Indiana
    "KY": 3870,  # Kentucky
    "OH": 155_123,  # Ohio
    # Northeast region:
    "ME": 15829,  # Maine
    "NH": 4546,  # New Hampshire
    "RI": 17000,  # Rhode Island
    "NJ": 21154,  # New Jersey
    "NY": 160_917,  # New York
    "PA": 65108,  # Pennsylvania
    "CT": 26_187,  # Connecticut
    "MA": 37087,  # Massachusetts
    "VA": 17667,  # Virginia
    "WV": 9168,  # West Virginia
    "MD": 19284,  # Maryland
    "VT": 2928,  # Vermont
    "DE": 3044,  # Delaware
    # Southeast region:
    "TN": 42479,  # Tennessee
    "MS": 15_205,  # Mississippi
    "AL": 15835,  # Alabama
    "GA": 29559,  # Georgia
    "FL": 46405,  # Florida
    "SC": 5680,  # South Carolina
    "NC": 28_348,  # North Carolina
}
S = state_demand  # Sales

print("Total states number: ", len(S))
print("Total demand: ", sum(S.values()))
print("Each state represents a demand center")

# TODO: falta alguém ler e verificar os dados (o input foi feito de forma manual)

# NOTE: Valor total deve ser 1_477_026, então precisa ajustar NY (que nao tem na foto) pra bater o valor final.

Total states number:  48
Total demand:  1477026
Each state represents a demand center


In [30]:
demand_centers = [f"{s}" for s in state_demand.keys()]

# pprint(demand_centers)

print(
    """
    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    """
)


    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    


In [31]:
# 2.2 Plant current capacities C[p]
C = {
    "Covington, KY": 620_000,
    "New York, NY": 430_000,
    "Arlington, TX": 300_000,
    "Long Beach, CA": 280_000,
}

# Plant stocking capacities C'[p]
C_prime = {
    "Covington, KY": 450_000,
    "New York, NY": 380_000,
    "Arlington, TX": 140_000,
    "Long Beach, CA": 180_000,
}

In [32]:
# 2.3 Unit production cost v[p] (variable production cost)
rho = {
    "Covington, KY": 21.0,
    "New York, NY": 19.9,
    "Arlington, TX": 21.6,
    "Long Beach, CA": 21.1,
}

In [33]:
# 2.4 Distance matrices (in miles)
#     d_pw[p][w] = distance plant → warehouse
#     d_ws[w][d] = distance warehouse → demand center
#     d_pd[p][d] = distance plant → demand center

df_pw = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_pw", index_col=0
)
df_ws = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_ws", index_col=0
)
df_pd = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_pd", index_col=0
)

d_pw = df_pw.to_dict()
d_ws = df_ws.to_dict()
d_pd = df_pd.to_dict()

In [34]:
# print("Distance matrices:")
# print("d_pw (plant to warehouse):")
# pprint(d_pw)
# print("d_wd (warehouse to demand center):")
# pprint(d_ws)
# print("d_pd (plant to demand center):")
# pprint(d_pd)

In [35]:
# 2.5 Inbound cost c_in[p][w] = 0.92 + 0.0034*d_pw
c_in = {p: {w: 0.92 + 0.0034 * d_pw[p][w] for w in warehouses} for p in plants}

# c_in

In [36]:
# Ler tabela 3 do excel
table3 = pd.read_excel(
    "../misc/tabelas.xlsx",
    sheet_name="table3",
)
table3.head()

,Warehouse No.,Storage ($/$),Handling ($/cwt),Stock Order Processing ($/order),Stock Order Size,Customer Order Processing ($/order),Customer Order size (cwt/order),Local delivery rate ($/cwt)
0,1,0.0672,0.46,18,400,1.79,9.05,1.90
1,2,0.0567,0.54,18,400,1.74,10.92,3.89
2,3,0.0755,0.38,18,400,2.71,11.59,2.02
3,4,0.0735,0.59,18,400,1.74,11.30,4.31
4,5,0.0946,0.50,18,401,0.83,9.31,1.89


In [37]:
# 2.6 Outbound cost c_out[w][j]:
local_rate = {i: v for i, v in enumerate(table3["Local delivery rate ($/cwt)"])}

c_out = {}
for i, w in enumerate(warehouses):
    c_out[w] = {}
    for s in demand_centers:
        d = d_ws[w][s]
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            c_out[w][s] = local_rate[i]
        else:
            # else use 5.45 + 0.0037*d
            c_out[w][s] = 5.45 + 0.0037 * d

# c_out

In [38]:
# custos de transporte da fabrica direto para o demand center.
c_out_prime = {}

for p in plants:
    c_out_prime[p] = {}
    for j in demand_centers:
        try:
            d = d_pd[p][j]
        except KeyError:
            print(f"{p} or {j} not found in d_pd")
            break
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            # c_out_prime[p][j] = local_rate[p]
            c_out_prime[p][j] = 0

            # TODO: verify this.
        else:
            # else use 5.45 + 0.0037*d
            c_out_prime[p][j] = 5.45 + 0.0037 * d

In [39]:
# Quanto representa 1 cwt. em $? Dividimos as vendas totais ($) pela demand total (cwt)
Gamma = 160_000_000 / 1_477_026  # ($ / cwt)
Gamma

108.32578438023434

In [40]:
# Custo de estocagem do armazém w
tau = {}

for i, w in enumerate(warehouses):
    # Fiz o casting para evitar o np.float64 nos prints, mas nao precisava
    tau[w] = float(table3["Storage ($/$)"][i])

In [41]:
# Custo de handling do armazém w (epsilon)

epsilons = {}

for i, w in enumerate(warehouses):
    epsilons[w] = float(table3["Handling ($/cwt)"][i])

In [42]:
# gamma

gammas = {}

for i, w in enumerate(warehouses):
    gammas[w] = float(table3["Stock Order Processing ($/order)"][i])

In [43]:
# delta

deltas = {}

for i, w in enumerate(warehouses):
    deltas[w] = float(table3["Stock Order Size"][i])

In [44]:
# omega

omegas = {}

for i, w in enumerate(warehouses):
    omegas[w] = float(table3["Customer Order size (cwt/order)"][i])

In [45]:
# phi

phi = {}
for i, w in enumerate(warehouses):
    phi[w] = float(table3["Customer Order Processing ($/order)"][i])

## 3. Defining the model

In [46]:
m = Model(name="UsemoreWarehousing")

### Decision variables

In [47]:
Z = m.addVars(warehouses, vtype=GRB.BINARY, name="z")
X = m.addVars(plants, warehouses, vtype=GRB.CONTINUOUS, name="x")
Y = m.addVars(warehouses, demand_centers, vtype=GRB.CONTINUOUS, name="y")
W = m.addVars(plants, demand_centers, vtype=GRB.CONTINUOUS, name="w")

## 4. Objective Function

### Define all the costs components

In [48]:
# inbound transport (from plants to warehouses)
inbound__transport_cost = gurobipy.quicksum(
    c_in[i][j] * X[i, j] for i in plants for j in warehouses
)

# outbound transport (from warehouses to demand nodes)
outbound_transport_cost = gurobipy.quicksum(
    c_out[j][k] * Y[j, k] for j in warehouses for k in demand_centers
)

# transport cost from plants to demand centers
direct_transport_cost = gurobipy.quicksum(
    c_out_prime[i][k] * W[i, k] for i in plants for k in demand_centers
)

# production cost
# production_costs = gurobipy.quicksum(
#     rho[p] * (X[p, w] + W[p, d])
#     for p in plants
#     for w in warehouses
#     for d in demand_centers
# )
production_costs1 = gurobipy.quicksum(
    rho[p] * (X[p, w]) for p in plants for w in warehouses
)
production_costs2 = gurobipy.quicksum(
    rho[p] * (W[p, d]) for p in plants for d in demand_centers
)

# storage cost in warehouses
storage_costs = gurobipy.quicksum(
    X[p, w] * Gamma * tau[w] for p in plants for w in warehouses
)

# custo de handling nos armazéns
handling_costs = gurobipy.quicksum(
    2 * Z[w] * epsilons[w] * X[p, w] for p in plants for w in warehouses
)

# custo de processamento do pedido de estoque
stock_order_processing_costs = gurobipy.quicksum(
    X[p, w] * (gammas[w] / deltas[w]) * Z[w] for p in plants for w in warehouses
)

# custo de processamento do pedido do cliente
customer_order_processing_costs = gurobipy.quicksum(
    Y[w, d] * (phi[w] / omegas[w]) * Z[w] for w in warehouses for d in demand_centers
)

#### Define the objective function

In [49]:
m.setObjective(
    inbound__transport_cost
    + outbound_transport_cost
    + direct_transport_cost
    + production_costs1
    + production_costs2
    + storage_costs
    + handling_costs
    + stock_order_processing_costs
    + customer_order_processing_costs,
    sense=GRB.MINIMIZE,
)

## Constraints

In [50]:
# Demand satisfaction (OK)

for d in demand_centers:
    m.addConstr(
        gurobipy.quicksum(Y[w, d] for w in warehouses)
        + gurobipy.quicksum(W[p, d] for p in plants)
        == S[d],
        name=f"demand_{d}",
    )

In [51]:
# Plant production capacity (OK)

for p in plants:
    m.addConstr(
        gurobipy.quicksum(X[p, w] for w in warehouses)
        + gurobipy.quicksum(W[p, d] for d in demand_centers)
        <= C[p],
        name=f"plantCap_{p}",
    )

In [52]:
# restrição dos 10400 (OK)

for w in warehouses:
    # Um armazém só pode ser aberto se houver demanda de pelo menos 10_400 cwt
    m.addConstr(
        gurobipy.quicksum(Y[w, d] for d in demand_centers) >= 10_400 * Z[w],
        name=f"minThroughput_{w}",
    )

In [53]:
# restrições doe armazenagem de cada planta (OK)

for p in plants:
    m.addConstr(
        gurobipy.quicksum(W[p, d] for d in demand_centers) <= C_prime[p],
    )

In [54]:
# Flow balance at each warehouse (OK)

for w in warehouses:
    # Tudo que entra no armazém deve ser igual a tudo que sai
    m.addConstr(
        gurobipy.quicksum(X[p, w] for p in plants)
        == gurobipy.quicksum(Y[w, d] for d in demand_centers),
        name=f"flowBalance_{w}",
    )

In [55]:
# Restrições de Z (só posso usar um armazém se ele estiver aberto) (OK)
for w in warehouses:
    for p in plants:
        # NOTE: the C[p] here is working as the big-M
        m.addConstr(
            X[p, w] <= C[p] * Z[w],
            name=f"openWarehouse_{w},{p}",
        )
    for d in demand_centers:
        # NOTE: the S[d] here is working as the big-M
        m.addConstr(
            Y[w, d] <= S[d] * Z[w],
            name=f"openWarehouse_{w},{d}",
        )

## Solver

In [56]:
# NOTE: precisou ser o formato .rlp pois algumas variáveis têm espaço
m.write("model-baseline.rlp")

In [57]:
m.params.TimeLimit = 300  # seconds

m.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 7730U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2432 rows, 2524 columns and 11884 nonzeros
Model fingerprint: 0xd9a267eb
Model has 2288 quadratic objective terms
Variable types: 2480 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [8e-01, 1e+02]
  QObjective range [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 6e+05]
Found heuristic solution: objective 5.809556e+07
Presolve time: 0.01s
Presolved: 4720 rows, 4812 columns, 18748 nonzeros
Variable types: 4768 continuous, 44 integer (44 binary)

Root relaxation: objective 4.381531e+07, 343 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 

In [58]:
m.printStats()


Statistics for model UsemoreWarehousing:
  Linear constraint matrix    : 2432 Constrs, 2524 Vars, 11884 NZs
  Variable types              : 2480 Continuous,
44 Integer (44 Binary)
  Matrix coefficient range    : [ 1, 620000 ]
  Objective coefficient range : [ 0.8, 138.191 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 1004, 620000 ]


In [59]:
m.printQuality()


Solution quality statistics for model UsemoreWarehousing :
  Maximum violation:
    Bound       : 0.00000000e+00
    Constraint  : 0.00000000e+00
    Integrality : 0.00000000e+00


## Visualize results

### Costs

In [60]:
# Print the solution objective value
print(f"Objective function final value: ${m.objVal:,.2f}")

Objective function final value: $43,815,306.73


In [61]:
def display_cost_components():
    """Calculate and display all cost components"""

    # Get cost values from the model
    costs = {
        "Inbound transport cost": inbound__transport_cost.getValue(),
        "Outbound transport cost": outbound_transport_cost.getValue(),
        "Direct transport cost": direct_transport_cost.getValue(),
        "Production costs": production_costs1.getValue() + production_costs2.getValue(),
        "Storage costs": storage_costs.getValue(),
        "Handling costs": handling_costs.getValue(),
        "Stock order processing costs": stock_order_processing_costs.getValue(),
        "Customer order processing costs": customer_order_processing_costs.getValue(),
    }

    # Calculate total cost
    total = sum(costs.values())

    # Print the cost components in a formatted table
    print("\n" + "=" * 65)
    print(f"{'COST BREAKDOWN':^65}")
    print("=" * 65)
    print(f"{'Cost Component':<30} {'Amount ($)':>18} {'Percentage':>12}")
    print("-" * 65)

    for name, value in costs.items():
        percentage = (value / total) * 100
        print(f"{name:<32} ${value:>17,.2f} {percentage:>11.2f}%")

    print("-" * 65)
    print(f"{'TOTAL COST':<32} ${total:>17,.2f} {100:>11.2f}%")
    print("=" * 65)


# Call the function to display costs
display_cost_components()


                         COST BREAKDOWN                          
Cost Component                         Amount ($)   Percentage
-----------------------------------------------------------------
Inbound transport cost           $       663,067.42        1.51%
Outbound transport cost          $     2,016,312.34        4.60%
Direct transport cost            $     7,674,679.37       17.52%
Production costs                 $    30,710,761.60       70.09%
Storage costs                    $     2,332,626.62        5.32%
Handling costs                   $       325,578.86        0.74%
Stock order processing costs     $        24,276.63        0.06%
Customer order processing costs  $        68,003.90        0.16%
-----------------------------------------------------------------
TOTAL COST                       $    43,815,306.73      100.00%


### Locations

In [62]:
open_w = [w for w in warehouses if Z[w].X > 0.1]
closed_w = [w for w in warehouses if Z[w].X < 0.1]

print(f"Number of open warehouses 🏭: {len(open_w)} ")
print(f"Number of closed warehouses 🔒: {len(closed_w)} ")

# Calculate percentage of open warehouses
percentage_open = (len(open_w) / len(warehouses)) * 100
print(f"Percentage of warehouses open: {percentage_open:.1f}% 📊")

print("\n--- Warehouse openings ---")
for w in open_w:
    print(f" Open warehouse at {w}")

if len(closed_w) > 0:
    print("\n--- Warehouse closings ---")
    for w in closed_w:
        print(f" Closed warehouse at {w}")

Number of open warehouses 🏭: 4 
Number of closed warehouses 🔒: 40 
Percentage of warehouses open: 9.1% 📊

--- Warehouse openings ---
 Open warehouse at Atlanta
 Open warehouse at Boston
 Open warehouse at Chicago
 Open warehouse at Houston

--- Warehouse closings ---
 Closed warehouse at Buffalo
 Closed warehouse at Cleveland
 Closed warehouse at Davenport
 Closed warehouse at Detroit
 Closed warehouse at Grand Rapids
 Closed warehouse at Greensboro
 Closed warehouse at Kansas City
 Closed warehouse at Baltimore
 Closed warehouse at Memphis
 Closed warehouse at Milwaukee
 Closed warehouse at Orlando
 Closed warehouse at Pittsburgh
 Closed warehouse at Portland
 Closed warehouse at W Sacramento
 Closed warehouse at W Chester
 Closed warehouse at Albuquerque
 Closed warehouse at Billings
 Closed warehouse at Denver
 Closed warehouse at El Paso
 Closed warehouse at Camp Hill
 Closed warehouse at Las Vegas
 Closed warehouse at Minneapolis
 Closed warehouse at New Orleans
 Closed warehouse 

### Production

In [63]:
plant_production = {}

for p in plants:
    plant_production[p] = sum(X[p, w].X for w in warehouses if Z[w].X > 0.1) + sum(
        W[p, d].X for d in demand_centers if d in S and S[d] > 0 and W[p, d].X > 0.1
    )

print("\n------------------ Plant production ----------------------")
total_production = sum(plant_production.values())
for p in plants:
    percentage = (plant_production[p] / total_production) * 100
    print(
        f" Plant {p:>15} production: {plant_production[p]:,.2f} cwt ({percentage:>4.1f}%)"
    )


------------------ Plant production ----------------------
 Plant   Covington, KY production: 620,000.00 cwt (42.0%)
 Plant    New York, NY production: 430,000.00 cwt (29.1%)
 Plant   Arlington, TX production: 247,026.00 cwt (16.7%)
 Plant  Long Beach, CA production: 180,000.00 cwt (12.2%)


### Flows

In [64]:
print("\n---------- Plant → Warehouse flows -------------")
for p, w in X.keys():
    if X[p, w].X > 1e-6:
        print(f" {p:>15} → {w:<16} = {X[p, w].X:>5.0f} cwt")


---------- Plant → Warehouse flows -------------
   Covington, KY → Atlanta          = 97479 cwt
   Covington, KY → Chicago          = 72521 cwt
    New York, NY → Boston           = 50000 cwt
   Arlington, TX → Houston          = 107026 cwt


In [65]:
print("\n---------- Plant → Demand flows ---")
for w, j in W.keys():
    if W[w, j].X > 1e-6:
        print(f" {w:>15} → {j}: {W[w, j].X:>7.0f} cwt")


---------- Plant → Demand flows ---
   Covington, KY → IA:    8438 cwt
   Covington, KY → MO:   19809 cwt
   Covington, KY → IL:   72839 cwt
   Covington, KY → MI:  103448 cwt
   Covington, KY → IN:   43994 cwt
   Covington, KY → KY:    3870 cwt
   Covington, KY → OH:  155123 cwt
   Covington, KY → TN:   42479 cwt
    New York, NY → MI:    1733 cwt
    New York, NY → ME:    2916 cwt
    New York, NY → NH:    4546 cwt
    New York, NY → RI:   17000 cwt
    New York, NY → NJ:   21154 cwt
    New York, NY → NY:  160917 cwt
    New York, NY → PA:   65108 cwt
    New York, NY → CT:   26187 cwt
    New York, NY → VA:   17667 cwt
    New York, NY → WV:    9168 cwt
    New York, NY → MD:   19284 cwt
    New York, NY → VT:    2928 cwt
    New York, NY → DE:    3044 cwt
    New York, NY → NC:   28348 cwt
   Arlington, TX → WA:   32437 cwt
   Arlington, TX → OR:    3236 cwt
   Arlington, TX → AZ:    9063 cwt
   Arlington, TX → ID:    7153 cwt
   Arlington, TX → UT:    9001 cwt
   Arlington, TX →

In [66]:
print("\n------- Warehouse → Demand flows ---")
for w, j in Y.keys():
    if Y[w, j].X > 1e-6:
        print(f" {w:>16} → {j}: {Y[w, j].X:>5.0f} cwt")


------- Warehouse → Demand flows ---
          Atlanta → AL: 15835 cwt
          Atlanta → GA: 29559 cwt
          Atlanta → FL: 46405 cwt
          Atlanta → SC:  5680 cwt
           Boston → ME: 12913 cwt
           Boston → MA: 37087 cwt
          Chicago → ND:  5703 cwt
          Chicago → MN:  5633 cwt
          Chicago → IA: 23737 cwt
          Chicago → WI: 37448 cwt
          Houston → TX: 76810 cwt
          Houston → LA: 15011 cwt
          Houston → MS: 15205 cwt
